In [ ]:
import pandas as pd
from cytoolz import identity
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import multiprocessing as mp

In [ ]:
df= pd.read_csv('../input/news-data-set-for-classification/classification.csv', index_col=0)
test= pd.read_csv('../input/news-data-set-for-classification/classification_test.csv', index_col=0)
nlp = spacy.load('en', disable=['tagger', 'ner', 'parser'])

In [ ]:
%%time

def tokenize(text):
    return [tok.text for tok in nlp.tokenizer(text.lower()) if (tok.text not in STOP_WORDS) and ((tok.text).isalpha())]

df['tokens'] = df['text'].apply(tokenize)
test['tokens'] = test['text'].apply(tokenize)

In [ ]:
from sklearn.feature_extraction.text import *
dtm = CountVectorizer(analyzer=identity)

X = dtm.fit_transform(df['tokens'])
X_test = dtm.transform(test['tokens'])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers

In [ ]:
modelLG = Sequential()
modelLG.add(Dense(1, input_shape=(X.shape[1],), activation='sigmoid'
                   #, kernel_regularizer=regularizers.l2(0.01),
                   #activity_regularizer=regularizers.l1(0.01)
                   ))


In [ ]:
%%time

modelLG.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
history = modelLG.fit(X, df['sports'],epochs=150, batch_size=500, verbose =0, validation_data=(X_test, test['sports']), use_multiprocessing=True)

In [ ]:
from matplotlib import pyplot as plt
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.ylim(.96, 1.01)
plt.axhline(y=1.0, color="purple", linestyle="--")
plt.legend()
plt.show()

By the way the graph can be saved as a picture file.

In [ ]:
plt.savefig('AccuracyDuringFitting.png')

In [ ]:
predictions = modelLG.predict_classes(X_test)

In [ ]:
def error_metrics(model_name, true_values, predictions, errors=None):
    if errors is None: 
        errors = pd.DataFrame({'metrics': ['accuracy', 'precision', 'recall', 'f1']})
        
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    accuracy = accuracy_score(true_values, predictions)
    precision = precision_score(true_values, predictions)
    recall = recall_score(true_values, predictions)
    f1 = f1_score(true_values, predictions)
    errors[model_name]= [accuracy,  precision, recall, f1]
    return errors

In [ ]:
err = error_metrics(model_name='Stochastic Logistic Regression', true_values = test['sports'], predictions=predictions)
err